Parallel Maps are Common
---------------------------

For simple operations like an embarrassingly parallel map (applying the same function to many inputs) we have *many* options.  They all perform about the same.

This notebook goes through the user interface of several of them on the same problem.

In [ ]:
from glob import glob
import ujson as json
import pandas as pd

In [ ]:
filenames = sorted(glob('../data/data-*.json'))


def load_parse_store(fn):
    with open(fn) as f:
        data = [json.loads(line) for line in f]
        
    df = pd.DataFrame(data)
    df.to_hdf(fn.replace('json', 'h5'), '/data')

### Sequential for loops

In [ ]:
%%time

for fn in filenames:
    load_parse_store(fn)

### Concurrent.futures

In [ ]:
%%time

from concurrent.futures import ProcessPoolExecutor
e = ProcessPoolExecutor()

list(e.map(load_parse_store, filenames))

### Multiprocessing

In [ ]:
%%time 

from multiprocessing import Pool
p = Pool()

list(p.map(load_parse_store, filenames))

### Joblib

In [ ]:
%%time 

from joblib import Parallel, delayed

result = Parallel(n_jobs=4, backend='multiprocessing')(delayed(load_parse_store)(fn) for fn in filenames)

### IPython Parallel

TODO: is there a nice way to set up IPyParallel?  Do we have them run `ipcluster` locally first?

### PySpark

In [ ]:
%%time

import pyspark

sc = pyspark.SparkContext('local[4]')

In [ ]:
%%time

rdd = sc.parallelize(filenames)
result = rdd.map(load_parse_store).collect()

### Dask.bag

In [ ]:
%%time

import dask.bag as db

b = db.from_sequence(filenames)
b.map(load_parse_store).compute()

### Dask.delayed

In [ ]:
%%time

from dask import delayed, compute
import dask.multiprocessing

compute(*[delayed(load_parse_store)(fn) for fn in filenames], 
        get=dask.multiprocessing.get)